# Pathology prediction (fracture)

@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the Atelectasis pathology using the CSV embeddings file



## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the Atelectasis pathology is : (N = 15,213)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.ATELECTASIS, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Atelectasis", experiment_name="CP_Atelectasis", filepath="./results/atelectasis")

# Model training
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='kfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-09-30 11:46:44,093	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=18548) Outer fold 1
(run_fold pid=18548) Configuring PyCaret for outer fold 1
(run_fold pid=18529) Outer fold 4 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Processing:  75%|███████▌  | 3/4 [04:48<01:46, 106.82s/it] [repeated 3x across cluster]
                                                          


(run_fold pid=18529)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=18529) Fold                                                          
(run_fold pid=18529) 0       0.9810  0.8703  0.9995  0.9814  0.9904  0.3216  0.4193
(run_fold pid=18529) 1       0.9748  0.7487  1.0000  0.9747  0.9872  0.2407  0.3699
(run_fold pid=18529) 2       0.9789  0.7530  0.9995  0.9793  0.9893  0.3426  0.4391
(run_fold pid=18529) 3       0.9810  0.7679  1.0000  0.9809  0.9904  0.3671  0.4741
(run_fold pid=18529) 4       0.9810  0.8105  1.0000  0.9809  0.9904  0.2970  0.4176
(run_fold pid=18529) Mean    0.9794  0.7901  0.9998  0.9795  0.9895  0.3138  0.4240
(run_fold pid=18529) Std     0.0024  0.0457  0.0003  0.0025  0.0012  0.0433  0.0339
(run_fold pid=18529) Configuring PyCaret for outer fold 4 [repeated 3x across cluster]
(run_fold pid=18529) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=18542) 0       0.9784  0.8330  1.0000  0.9784 

Processing:   0%|          | 0/4 [00:00<?, ?it/s]
                                                          [repeated 3x across cluster]


(run_fold pid=18533) Configuring PyCaret for outer fold 5


Processing:  25%|██▌       | 1/4 [00:01<00:03,  1.27s/it]


(run_fold pid=18529) Transformation Pipeline and Model Successfully Saved
(run_fold pid=18529)                        Model  Accuracy     AUC  ...     F1   Kappa     MCC
(run_fold pid=18529) 0  Extreme Gradient Boosting    0.9822  0.8288  ...  0.991  0.3805  0.4741
(run_fold pid=18529) 
(run_fold pid=18529) [1 rows x 8 columns]


Processing:  75%|███████▌  | 3/4 [04:33<01:40, 100.99s/it]
                                                          


(run_fold pid=18533)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=18533) Fold                                                          
(run_fold pid=18533) 0       0.9815  0.8252  1.0000  0.9814  0.9906  0.3940  0.4953
(run_fold pid=18533) 1       0.9789  0.7933  1.0000  0.9788  0.9893  0.3814  0.4854
(run_fold pid=18533) 2       0.9759  0.7660  1.0000  0.9758  0.9878  0.1422  0.2766
(run_fold pid=18533) 3       0.9805  0.7736  1.0000  0.9804  0.9901  0.3608  0.4691
(run_fold pid=18533) 4       0.9810  0.8219  0.9989  0.9819  0.9904  0.3211  0.4035
(run_fold pid=18533) Mean    0.9796  0.7960  0.9998  0.9797  0.9896  0.3199  0.4260
(run_fold pid=18533) Std     0.0020  0.0242  0.0004  0.0022  0.0010  0.0922  0.0813
(run_fold pid=18533) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=18542) Transformation Pipeline and Model Successfully Saved
(run_fold pid=18542)                        Model  Accuracy     AUC  ... 